In [1]:
import sys
sys.path.append('..')

# Formaty danych

Związki chemiczne na dysku najczęściej przechowywane są w formacie SMILES. Zwykle mamy do czynienia z plikami CSV, w których znajduje się kolumna o nazwie `smiles`, a w pozostałych kolumnach opis danego związku. Czasem wystarczy nam tylko SMILES i w tym przypadku plik często oznaczany jest rozszerzeniem `.smi`, a kolejne związki są zapisane w osobnych linijkach w pliku.

## Konformacje i pola siłowe

SMILES nie uwzględnia położenia atomów w przestrzeni 3D, czyli tzw. **konformacji** związku. Konformacje powstają przez obracanie się rotowalnych wiązań tak, aby uzyskać korzystną energetycznie pozycję. Związek może mieć więcej niż jedną stabilną konformację. Komputerowo jesteśmy w stanie dla małych związków uzyskać konformacje bardzo bliskie prawdziwym. Robi się to za pomocą **pól siłowych**, czyli metod optymalizacji, które stopniowo układają cząsteczkę w pozycji, która osiąga minimum energii.

## Formaty 3D dla związków chemicznych

Gdy potrzebujemy zachować związek wraz z konformacją, lepszym formatem danych od samego SMILES-a będzie MOL2 albo SDF. Formaty te pozwalają przechowywać informacje o pozycjach atomów, jak widoczne poniżej.

In [29]:
import os

import pandas as pd
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem

from ipywidgets import interact

In [5]:
zinc_df = pd.read_csv('../data/CC/CCAA.smi', sep=' ')
zinc_df

,smiles,zinc_id
0,CC(=O)Nc1ccc(S(=O)(=O)N(C)CCO)cc1,29138
1,CCOc1ccc(C(=O)N[C@@H](C(=O)O)[C@H](C)O)cc1,344024
2,CNC(=O)N[C@@H]1N(C)C(=O)N[C@H](C(C)C)C1(C)C,517650
3,Cc1oc2nc(SCC(=O)NCCO)nc(N)c2c1C,2259245
4,O=C(O)CCCNC1=NS(=O)(=O)c2ccccc21,2637193
...,...,...
114001,O=C(c1nc[nH]c1C(F)(F)F)N1CCc2nn[nH]c2C1,828942673
114002,CNC(=O)CNC(=O)[C@H]1Cc2cc(Cl)ccc2O1,25266002
114003,CCOC1CC(CC(=O)N(C)[C@H]2C[C@@H](C(N)=O)C2)C1,863358233
114004,CNc1ccccc1C(=O)NC[C@@]1(O)CCO[C@@H]1C,227184162


In [57]:
def optimize_conformation(mol):
    mol = Chem.AddHs(mol)  # Adds hydrogens to make optimization more accurate
    AllChem.EmbedMolecule(mol)  # Adds 3D positions
    AllChem.MMFFOptimizeMolecule(mol)  # Improves the 3D positions using a force-field method
    return mol


@interact
def show_molecule(compound_idx=(0, len(zinc_df), 1)):
    smiles = zinc_df.iloc[compound_idx].smiles
    if not smiles:
        return
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return
    mol = optimize_conformation(mol)
    mblock = Chem.MolToMolBlock(mol, kekulize=False)  # Produces a MOL block, which can be stored in a .mol file

    view = py3Dmol.view(width=800, height=500)
    view.addModel(Chem.MolToMolBlock(mol, kekulize=True), 'mol')
    view.setStyle({'model': -1}, {"stick": {'color': 'white', 'radius': 0.15}, "sphere": {'radius': .4}})
    view.zoomTo({'model': -1})
    view.show()
    
    print('-' * 100)
    print(mblock)
    

# Białka

Białka są makrocząsteczkami (są duże) pełniącymi w organizmie różne funkcje. Niektóre białka odpowiadają za metabolizm (głównie enzymy), inne pośredniczą w komunikacji z komórkami (receptory sprzężone z białkiem G), a inne jeszcze uczestniczą w transporcie jonów (kanały jonowe). Dlatego też różne białka stają się podstawowym celem dla małocząsteczkowych leków.

Białka są polimerami, to znaczy składają się z mniejszych elementów zwanych aminokwasami:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Amino_acids.png/924px-Amino_acids.png)

![](https://upload.wikimedia.org/wikipedia/commons/3/32/SIMPLIFIED_PEPTIDE_CHAIN.png)

Aminokwasy mają różne właściwości i wpływają na kształt (konformację) cząsteczki. Ponieważ białka mają budowę liniową i składają się z powtarzalnych bloków, możemy je zapisywać w formacie tekstowym jako sekwencję liter odpowiadających aminokwasom, np. w formacie FASTA. Jest to tzw. pierwszorzędowa struktura białka.

```
>6NL3_1|Chain A|Cytochrome c oxidase assembly factor 6 homolog|Homo sapiens (9606)
GSMAAPSMKERQVCWGARDEYWKCLDENLEDASQCKKLRSSFESSCPQQWIKYFDKRRDYLKFKEKFEAGQFEPSETTAKS
```

## Zwijanie białek

Konformacje białek są bardziej złożonym problemem niż konformacje małych cząsteczek. Liczba atomów oraz rotowalnych wiązań w białkach jest ogromna, więc programy do liczenia pól siłowych nie działają najlepiej dla białek. Białka w organizmie produkowane są sekwencyjnie i od razu ulegają zwijaniu w ściśle określonych warunkach fizjologicznych, często z pomocą innych białek, tzw. czaperonów. Dlatego efekt zwijania jest bardzo precyzyjny i deterministyczny mimo wielu punktów swobody.

Długo problem zwijania białek był nieosiągalny dla metod komputerowych i dotąd klasyczną metodą poznawania konformacji białek jest **krystalizacja**. Wykrystalizowane białka można następnie "sfotografować", aby uzyskać model 3D z dużą dokładnością. Obecnie metody komputerowe uzyskują już dokładność, o której jeszcze niedawno sądzono, że nie będzie nigdy możliwa. DeepMind stworzył algorytm **AlphaFold** oparty o uczenie głębokie, który jest w stanie przewidzieć struktury 3D z dokładnością 0.96 Å r.m.s.d.95 (kolejne rozwiązanie w konkursie **CASP14** miało wynik 2.8 Å r.m.s.d.95). DeepMind zaczął udostępniać wyniki przewidywań dla białek, których kryształy dotąd nie były znane ([AlphaFold Protein Structure Database](https://alphafold.ebi.ac.uk/)). Kod AlphaFolda został udostępniony na otwartym źródle, a przewidywania dla nowych sekwencji białkowych można dokonać samodzielnie przez udostępniony [notebook Colab](https://colab.sandbox.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb).

![](https://lh3.googleusercontent.com/J5RC0j1DeUWaNc5h6nGwJwsQSLUuTXINP6we2ymLJ_WUg9bH-hvfvI8WVFeghN-_YR69MryNK5O2rFcVNwz9PZePpBtLdwdshCGzLdM=w2048-rw-v1)

## Baza PDB

[PDB](https://www.rcsb.org/) jest bazą struktur białek, gdzie możemy znaleźć kryształy interesujących nas celów biologicznych. Najbardziej popularnym formatem struktur bialek jest właśnie .pdb, który wygląda tak, jak załączono poniżej ([CYP 3A4](https://www.rcsb.org/structure/6MA7)).

In [27]:
@interact
def show_protein(style=['balls & sticks', 'cartoon', 'surface']):
    view = py3Dmol.view(width=800, height=500, query='pdb:6MA7')
    if style == 'cartoon':
        view.setStyle({'model': -1}, {"cartoon": {'color': 'cyan'}})
    elif style == 'balls & sticks':
        view.setStyle({'model': -1}, {"stick": {'color': 'white', 'radius': 0.15}, "sphere": {'radius': .4}})
    elif style == 'surface':
        view.setStyle({'model': -1}, {})
        view.addSurface('VDW')

    view.zoomTo({'model': -1})
    view.show()

interactive(children=(Dropdown(description='style', options=('balls & sticks', 'cartoon', 'surface'), value='b…

```
...
ATOM      1  N   GLY A  26     -34.317   8.706 -14.755  1.00128.59           N  
ANISOU    1  N   GLY A  26    16562  13800  18498   2794    437   1597       N  
ATOM      2  CA  GLY A  26     -33.904   7.326 -14.563  1.00126.57           C  
ANISOU    2  CA  GLY A  26    16140  13908  18043   2542    375   1417       C  
ATOM      3  C   GLY A  26     -32.608   6.995 -15.277  1.00128.93           C  
ANISOU    3  C   GLY A  26    16532  14226  18228   2319    463   1607       C  
ATOM      4  O   GLY A  26     -32.168   5.852 -15.299  1.00117.11           O  
ANISOU    4  O   GLY A  26    14921  13024  16553   2131    417   1500       O  
ATOM      5  N   THR A  27     -31.996   8.010 -15.878  1.00144.25           N  
ANISOU    5  N   THR A  27    18681  15832  20295   2349    618   1906       N  
ATOM      6  CA  THR A  27     -30.716   7.829 -16.548  1.00149.54           C  
ANISOU    6  CA  THR A  27    19431  16468  20918   2141    773   2119       C  
ATOM      7  C   THR A  27     -29.647   8.652 -15.842  1.00161.94           C  
ANISOU    7  C   THR A  27    21063  17579  22889   1854    953   2076       C  
ATOM      8  O   THR A  27     -29.018   9.523 -16.456  1.00167.94           O  
ANISOU    8  O   THR A  27    21978  18006  23825   1843   1158   2384       O  
ATOM      9  CB  THR A  27     -30.818   8.232 -18.017  1.00142.27           C  
ANISOU    9  CB  THR A  27    18706  15556  19794   2424    845   2558       C  
ATOM     10  OG1 THR A  27     -31.069   9.640 -18.105  1.00145.88           O  
ANISOU   10  OG1 THR A  27    19354  15572  20501   2606    963   2775       O  
ATOM     11  CG2 THR A  27     -31.950   7.475 -18.695  1.00132.20           C  
ANISOU   11  CG2 THR A  27    17362  14727  18142   2738    591   2545       C  
...
HETATM 3735  CHA HEM A 601     -15.300 -19.651  -9.279  1.00 42.50           C  
ANISOU 3735  CHA HEM A 601     4140   6322   5688     50   -273   -757       C  
HETATM 3736  CHB HEM A 601     -16.180 -22.481 -13.088  1.00 45.45           C  
ANISOU 3736  CHB HEM A 601     4576   6920   5771    211     98   -793       C  
HETATM 3737  CHC HEM A 601     -16.854 -26.224 -10.126  1.00 35.75           C  
ANISOU 3737  CHC HEM A 601     3757   5260   4565    363    -24   -922       C  
HETATM 3738  CHD HEM A 601     -15.649 -23.507  -6.308  1.00 43.62           C  
ANISOU 3738  CHD HEM A 601     4865   6249   5459    446   -422   -814       C  
HETATM 3739  C1A HEM A 601     -15.509 -20.056 -10.578  1.00 42.15           C  
ANISOU 3739  C1A HEM A 601     4058   6356   5602     54   -122   -714       C  
...
```

# Molecular Docking

Dokowanie molekularne jest narzędziem ułatwiającym badanie interakcji między małą cząsteczką a jej celem białkowym. Na powierzchni białka widoczne są wgłębienia (**kieszenie wiążące**), w które może wejść cząsteczka (zwana też **ligandem**), aby modulować funkcję danego białka. Narzędzia do dokowania molekularnego mają na celu ułożenie liganda w najbardziej korzystnej energetycznie pozycji w kieszeni wiążącej białka. Zdefiniowana jest funkcja energii, która jest optymalizowana przez zmianę pozycji liganda w kieszeni i obracanie wiązaniami rotowalnymi. Konformacja uzyskana w tym procesie nazywana jest **konformacją aktywną**.

Dokowanie używane też jest w **skriningu wirtualnym** do znajdywania cząstek dobrze wiążących się z białkiem. Symulacja ta obarczona jest sporym błędem, ale jest bardzo skuteczna w odsiewaniu cząstek, które słabo się wiążą. Dobry wynik dokowania nie oznacza jeszcze aktywności związku, ale jest jej przybliżeniem.

**Zadanie:** Zadokuj związki do enzymu BACE. Czy docking score'y zgadzają się z aktywnością związków? Jak wyniki dokowania można użyć do wytrenowania lepszych modeli przewidujących aktywność?

In [37]:
def dock_molecule(mol):
    Chem.MolToMolFile(mol, f'molecule.mol')
    !obabel -imol molecule.mol -omol2 -O molecule.mol2
    os.remove(f'molecule.mol')
    
    # Dock the molecule stored in file molecule.mol2
    !smina -r ../data/bace/4ivt.pdb -l molecule.mol2 --center_x 22.312222 --center_y 23.862839 --center_z 0.34194124 --size_x 30 --size_y 30 --size_z 30 --exhaustiveness 32 --out molecule_docked.mol2

In [43]:
bace_path = 'https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/bace.csv'
df = pd.read_csv(bace_path)
df.head()

,mol,CID,Class,Model,pIC50,MW,AlogP,HBA,HBD,RB,...,PEOE6 (PEOE6),PEOE7 (PEOE7),PEOE8 (PEOE8),PEOE9 (PEOE9),PEOE10 (PEOE10),PEOE11 (PEOE11),PEOE12 (PEOE12),PEOE13 (PEOE13),PEOE14 (PEOE14),canvasUID
0,O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2c...,BACE_1,1,Train,9.154901,431.56979,4.4014,3,2,5,...,53.205711,78.640335,226.85541,107.43491,37.133846,0.000000,7.980170,0.0,0.000000,1
1,Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(...,BACE_2,1,Train,8.853872,657.81073,2.6412,5,4,16,...,73.817162,47.171600,365.67694,174.07675,34.923889,7.980170,24.148668,0.0,24.663788,2
2,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,BACE_3,1,Train,8.698970,591.74091,2.5499,4,3,11,...,70.365707,47.941147,192.40652,255.75255,23.654478,0.230159,15.879790,0.0,24.663788,3
3,S1(=O)(=O)C[C@@H](Cc2cc(O[C@H](COCC)C(F)(F)F)c...,BACE_4,1,Train,8.698970,591.67828,3.1680,4,3,12,...,56.657166,37.954151,194.35304,202.76335,36.498634,0.980913,8.188327,0.0,26.385181,4
4,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,BACE_5,1,Train,8.698970,629.71283,3.5086,3,3,11,...,78.945702,39.361153,179.71288,220.46130,23.654478,0.230159,15.879790,0.0,26.100143,5


In [56]:
smiles = ...
mol = Chem.MolFromSmiles(smiles)
dock_molecule(mol)

In [52]:
# Score only to see the scoring function components
!smina -r ../data/bace/4ivt.pdb -l molecule_docked.mol2 --score_only

In [55]:
with open(f"../data/bace/4ivt.pdb") as ifile:
    system = "".join([x for x in ifile])

if os.path.exists(f"molecule_docked.mol2"):
    with open(f"molecule_docked.mol2") as ifile:
        mol = "".join([x for x in ifile])
        mols = ['@<TRIPOS>MOLECULE' + conf for conf in mol.split('@<TRIPOS>MOLECULE') if conf]
else:
    mols = None

view = py3Dmol.view(width=800, height=500)
view.addModelsAsFrames(system)
view.setStyle({'model': -1}, {"cartoon": {'color': 'cyan'}})
if mols:
    view.addModel(mols[0], 'mol')
    view.setStyle({'model': -1}, {"stick": {'color': 'white', 'radius': 0.15}, "sphere": {'radius': .4}})
# view.addBox({'center': {'x': pocket_center[0], 'y': pocket_center[1], 'z': pocket_center[2]}, 'dimensions': {'w': 30, 'h': 30, 'd': 30}, 'wireframe': True})
view.zoomTo({'model': -1})
view.show()

## Kolejne kroki

Mając zadokowane związki możemy badań interakcje między związkiem chemicznym a białkiem. Można do tego użyć pakietu [ProLIF](https://prolif.readthedocs.io/en/latest/notebooks/visualisation.html). Możemy też w uczeniu modeli użyć **fingerprintów oddziaływań**, które opisują, z którymi aminokwasami dany ligand oddziałuje w kieszeni wiążącej.

Innym częstym zastosowaniem kształtu 3D związku jest obliczanie **farmakoforów**, czyli cech w przestrzeni 3D, które są kluczowe dla interakcji z białkiem. Znajdując kluczowe interakcje aktywnych związków buduje się hipotezę farmakoforową, która może być użyta do przefiltrowania dużych baz związków w poszukiwaniu kolejnych hitów.